In [1]:
import sys,os

sys.path.append("..")
import modeling

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tensorflow as tf
bert_config_file = "/data/xuht/bert/chinese_L-12_H-768_A-12/bert_config.json"
init_checkpoint = "/data/xuht/bert/chinese_L-12_H-768_A-12/bert_model.ckpt"
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
graph = tf.Graph()

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

with graph.as_default():
    sess = tf.Session()
    input_ids = tf.placeholder(tf.int32, 
                                    [None, 128],
                                    name="input_ids")
    input_mask = tf.cast(input_ids, tf.bool)
    segment_ids = tf.placeholder(tf.int32, 
                                [None, 128],
                                name="segment_ids")
    mode = tf.estimator.ModeKeys.TRAIN
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    model = modeling.BertModel(
          config=bert_config,
          is_training=True,
          input_ids=input_ids,
          input_mask=input_mask,
          token_type_ids=segment_ids,
          use_one_hot_embeddings=False)
    tvars = tf.trainable_variables()
    (assignment_map,
       initialized_variable_names) = modeling.get_assigment_map_from_checkpoint(
           tvars, init_checkpoint)
    
    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
    for var in tvars:
        init_string = ""
        if var.name in initialized_variable_names:
            init_string = ", *INIT_FROM_CKPT*"
        tf.logging.info(" name = %s, shape = %s%s", var.name, var.shape,
                                        init_string)
    
    def gather_indexes(sequence_tensor, positions):
        """Gathers the vectors at the specific positions over a minibatch."""
        sequence_shape = modeling.get_shape_list(sequence_tensor, expected_rank=3)
        batch_size = sequence_shape[0]
        seq_length = sequence_shape[1]
        width = sequence_shape[2]

        flat_offsets = tf.reshape(
                tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
        flat_positions = tf.reshape(positions + flat_offsets, [-1])
        flat_sequence_tensor = tf.reshape(sequence_tensor,
                                                                            [batch_size * seq_length, width])
        output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
        return output_tensor
    filename = "/data/xuht/eventy_detection/bert/guangfa/bert/pretrain_finicial_new.tfrecords"
    max_seq_length = 128
    max_predictions_per_seq = 20
    name_to_features = {
        "input_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "input_mask":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "segment_ids":
            tf.FixedLenFeature([max_seq_length], tf.int64),
        "masked_lm_positions":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_ids":
            tf.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_weights":
            tf.FixedLenFeature([max_predictions_per_seq], tf.float32),
        "next_sentence_labels":
            tf.FixedLenFeature([1], tf.int64),
    }
    
    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)
        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example
    
    def tfrecord_train_input_fn():
        tfrecord_dataset = tf.data.TFRecordDataset(filename)
        tfrecord_dataset = tfrecord_dataset.map(lambda   x:_decode_record(x, name_to_features)).shuffle(True).batch(2)
        tfrecord_dataset = tfrecord_dataset.repeat(1)
        tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()

        return tfrecord_iterator.get_next()
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    a = tfrecord_train_input_fn()
    p = tf.placeholder(tf.float32, [None, None])
    ll = tf.placeholder(tf.int32, [None, None])
    lll = {"output":tf.less(p.get_shape()[-1], 512)}
    ss = sess.run(lll, feed_dict={p:np.random.random((3,4))})
    print(ss)
    
    for l in range(1):
        label = tf.reshape(r["masked_lm_ids"], [-1])
        r1, r2 = sess.run([a, label])
#         print(r["masked_lm_positions"].shape)
#         import numpy as np
#         t = np.random.random((1,128,64)).astype(np.float32)
#         t = tf.constant(t)
#         ss = gather_indexes(t, a["masked_lm_positions"])
#         r = sess.run([ss])
        for key in r1:
            print(r1[key].shape, key, r1)
        print(r2.shape)

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [47]:
from sklearn.metrics import precision_score
y_true = [0, 1, 1, 0, 1, 1,0]
y_pred = [1, 1, 1, 1, 1, 1,1]
precision_score(y_true, y_pred, average='macro', labels=[1]) 

0.5714285714285714

In [48]:
precision_score(y_true, y_pred, average='micro', labels=[1]) 

0.5714285714285714

In [45]:
def abc(a,b):
    def bc(c,d):
        print(c,d)
    return [_, bc]

In [46]:
d = abc(1,2)
d[1](3,4)

3 4


In [4]:
with graph.as_default():
    max_seq_length = 50
    name_to_features = {
            "input_ids":
                    tf.FixedLenFeature([max_seq_length], tf.int64),
            "input_mask":
                    tf.FixedLenFeature([max_seq_length], tf.int64),
            "segment_ids":
                    tf.FixedLenFeature([max_seq_length], tf.int64),
            "label_ids":
                    tf.FixedLenFeature([1], tf.int64),
    }
    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)
        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example
    filename = "/data/xuht/eventy_detection/sentiment/model/test/train.tfrecords"
#     filename = "/data/xuht/bert/chinese_L-12_H-768_A-12/xnli_output/eval.tf_record"
    def tfrecord_train_input_fn(decode_fn):
        tfrecord_dataset = tf.data.TFRecordDataset(filename)
        tfrecord_dataset = tfrecord_dataset.map(lambda   x:decode_fn(x, name_to_features)).shuffle(True)#.batch(20)
        tfrecord_dataset.batch(100)
        tfrecord_dataset.repeat(1)
#         tfrecord_dataset = tfrecord_dataset.repeat(1)
        tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()

        return tfrecord_iterator.get_next()
    
    iterator = tfrecord_train_input_fn(_decode_record)
    
    for l in range(1):
        print(sess.run(iterator)["label_ids"].shape)

(1,)


In [5]:
import sys,os
sys.path.append("..")

import numpy as np
import tensorflow as tf
from example import bert_classifier
from bunch import Bunch
from example import feature_writer, write_to_tfrecords, classifier_processor
from data_generator import tokenization
from data_generator import tf_data_utils
from model_io import model_io

ImportError: No module named 'example'

In [ ]:
data = pd.read_excel("/data/xuht/eventy_detection/data_1114.xlsx")

sent_key = {}
for index in range(data.shape[0]):
    if data.loc[index]["句子内容"] in sent_key:
        sent_key[data.loc[index]["句子内容"]].append(data.loc[index]["预期事件"])
    else:
        sent_key[data.loc[index]["句子内容"]] = [data.loc[index]["预期事件"]]
        
real_sent = {}
for key in sent_key:
    if len(set(sent_key[key])) >= 2:
        continue
    else:
        real_sent[key] = list(set(sent_key[key]))[0]
        
data_set = [key for key in real_sent]
data_label = [real_sent[key] for key in real_sent]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_set, data_label, test_size=0.2, random_state=42)
with open("/data/xuht/eventy_detection/event/model/sentiment_11_14/train.txt", "w") as fwobj:
    for item, label in zip(X_train, y_train):
        fwobj.write(" ".join([item, "__label__{}".format(str(label))])+"\n")
with open("/data/xuht/eventy_detection/event/model/sentiment_11_14/test.txt", "w") as fwobj:
    for item, label in zip(X_test, y_test):
        fwobj.write(" ".join([item, "__label__{}".format(str(label))])+"\n")

label2id = {}
id2label = {}
label = list(set(data_label))
for index, l in enumerate(label):
    label2id[l] = index
    id2label[index] = l

with open("/data/xuht/eventy_detection/event/model/test/label2id.json", "w") as fwobj:
    json.dump({"label2id":label2id, "id2label":id2label}, fwobj)